## Run Harmonic Regression on Tesseract

In [1]:
import geodesic
import geodesic.tesseract as T
import numpy as np
from datetime import datetime

/home/rob/miniconda3/envs/har-reg/lib/python3.8/site-packages/geodesic/utils/downloader.py:36: UserWarning: google.cloud.storage is installed, but GOOGLE_APPLICATION_CREDENTIALS is not set
  warnings.warn(


In [2]:
proj = geodesic.set_active_project("har-reg")

In [3]:
landsat = geodesic.get_dataset("landsat-8")
print(landsat)

dataset:earth-bservation:satellite:electro-optical:landsat-8


In [4]:
bbox = (-94.691162,44.414164,-94.218750,44.676466) # Southern Minnesota
end_date = datetime.now()
end_date

datetime.datetime(2023, 10, 18, 14, 6, 6, 659377)

In [5]:
job = T.Job(
    name="har-reg-demo",
    alias="Harmonic Regression Demo",
    description="Demo of harmonic regression model",
    project=proj,
    bbox=bbox,
    bbox_epsg=4326,
    output_epsg=3857,
)

In [6]:
job.add_create_assets_step(
    name="add-landsat",
    asset_name="landsat",
    dataset="landsat-8",
    dataset_project=proj,
    asset_bands=[
        {"asset": "SR_B2", "bands": [0]},
        {"asset": "SR_B3", "bands": [0]},
        {"asset": "SR_B4", "bands": [0]},
        {"asset": "SR_B5", "bands": [0]},
        {"asset": "SR_B6", "bands": [0]},
        {"asset": "SR_B7", "bands": [0]},
    ],
    output_time_bins=dict(
        strided=T.StridedBinning(
            from_selection=T.BinSelection(datetime=end_date),
            duration="8D",
            stride="8D",
            count=200
        )
    ),
    output_bands=["blue", "green", "red", "nir", "swir1", "swir2"],
    chip_size=512,
    pixel_dtype=np.uint16,
    pixels_options=dict(
        pixel_size=(30.0, 30.0),
    ),
    fill_value="nan",
    workers=5,
);

In [7]:
job.add_model_step(
    name="run-har-reg",
    container=T.Container(
        repository="us-central1-docker.pkg.dev/double-catfish-291717/seerai-docker/images/",
        image="har-reg",
        tag="v0.0.2",
        args={"forder": 4}
    ),
    inputs=[T.StepInput(
        asset_name="landsat",
        dataset_project=proj,
        spatial_chunk_shape=(512, 512),
        type="tensor",
        time_bin_selection=T.BinSelection(all=True),
    )],
    outputs=[
        T.StepOutput(
            asset_name="brightness-params",
            chunk_shape=(1, 10, 256, 256),
            type="tensor",
            pixel_dtype="<f2",
            fill_value="0.0",
        ),
        T.StepOutput(
            asset_name="greenness-params",
            chunk_shape=(1, 10, 256, 256),
            type="tensor",
            pixel_dtype="<f2",
            fill_value="0.0",
        ),
        T.StepOutput(
            asset_name="wetness-params",
            chunk_shape=(1, 10, 256, 256),
            type="tensor",
            pixel_dtype="<f2",
            fill_value="0.0",
        )
    ],
    workers=5
);

In [8]:
res = job.submit(dry_run=True)
res

/home/rob/miniconda3/envs/har-reg/lib/python3.8/site-packages/geodesic/descriptors.py:102: UserWarning: field global_properties is deprecated and will be removed in a future version
  warnings.warn(
